In [ ]:
import requests
from urllib import parse
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import time

api_key = 'RGAPI-7cca0c8b-aba6-42fe-bb78-3c3081f5714a'

count = 40


request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

#   tft_v1_summoner_data('GOLD','I')
def tft_v1_summoner_userList(tier,division,page):
    url=f"https://kr.api.riotgames.com/tft/league/v1/entries/{tier}/{division}?page=1&api_key={api_key}"
    return requests.get(url,headers = request_header).json()

def tft_v1_summoner_puuid_by_summonerId(summonerId):
    url=f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={api_key}"
    puuid = requests.get(url,headers = request_header).json()
    return puuid['puuid']

def tft_v1_matchid_by_puuid(puuid,count):
    url=f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&startTime=0&count={count}&api_key={api_key}"
    return requests.get(url,headers=request_header).json()

def tft_v1_matchdata_by_matchid(matchid):
    url=f"https://asia.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={api_key}"
    matchdata = requests.get(url,headers=request_header).json()
    for user_data in matchdata["info"]['participants']:
        final_data = []        
        user = []
        for augments in user_data['augments']:
                user.append(augments)
        for i in range(3-len(user_data['augments'])):
            user.append('noAugments')
                
        user.append(user_data['placement'])
        for unit in user_data['units']:
            list=[]
            list.append(unit['character_id'])
            list.append(unit['tier'])
            for item in unit['itemNames']:
                list.append(item)
            for i in range(3-len(unit['itemNames'])):
                list.append('noItems')
            user.append(list)
        final_data.append(user)
    return final_data

for tier in ['DIAMOND','PLATINUM','GOLD','SILVER','BRONZE','IRON']: #   ['DIAMOND','PLATINUM','GOLD','SILVER','BRONZE','IRON']
    for division in ['I','II','III','IIII'] :
        final_result = []
        print(tier,division)
        i = 1
        while i<=10: # 탐색할 티어당 페이지 수
            summoner_userList = tft_v1_summoner_userList(tier,division,i)
            for summoner_user in tqdm(summoner_userList):
                try:
                    puuid = tft_v1_summoner_puuid_by_summonerId(summoner_user['summonerId'])
                except KeyError:
                    print('error here')
                    pass
                find_matchid = tft_v1_matchid_by_puuid(puuid,count) #(puuid,count) 푸이그당 전적을 긁어올 개수
                for matchid in find_matchid:
                    try:
                        matchdatas = tft_v1_matchdata_by_matchid(matchid)
                        final_result+=matchdatas
                    except:
                        time.sleep(0.25)
            df = pd.DataFrame(final_result)
            df.to_csv(f'data/battle_data/{tier}_{division}_{i}_data.csv')            
            if len(summoner_userList) == 0:
                break
            elif i >=150:
                break
            else:
                i+=1
            

In [1]:


summonerlist=['TFT6_Augment_Ascension', 'TFT6_Augment_BandOfThieves2', 'TFT6_Augment_BinaryAirdrop', 'TFT6_Augment_BlueBattery2', 'TFT6_Augment_CalculatedLoss', 'TFT6_Augment_CelestialBlessing1', 'TFT6_Augment_CelestialBlessing2', 'TFT6_Augment_CelestialBlessing3', 'TFT6_Augment_ClearMind', 'TFT6_Augment_ComponentGrabBag', 'TFT6_Augment_CyberneticImplants1', 'TFT6_Augment_CyberneticImplants2', 'TFT6_Augment_CyberneticImplants3', 'TFT6_Augment_CyberneticShell1', 'TFT6_Augment_CyberneticShell2', 'TFT6_Augment_CyberneticShell3', 'TFT6_Augment_CyberneticUplink1', 'TFT6_Augment_CyberneticUplink2', 'TFT6_Augment_CyberneticUplink3', 'TFT6_Augment_Distancing', 'TFT6_Augment_Distancing2', 'TFT6_Augment_Distancing3', 'TFT6_Augment_Diversify1', 'TFT6_Augment_Diversify2', 'TFT6_Augment_Diversify3', 'TFT6_Augment_Electrocharge1', 'TFT6_Augment_Electrocharge2', 'TFT6_Augment_Electrocharge3', 'TFT6_Augment_Featherweights1', 'TFT6_Augment_Featherweights2', 'TFT6_Augment_Featherweights3', 'TFT6_Augment_FirstAidKit', 'TFT6_Augment_ForceOfNature', 'TFT6_Augment_FuturePeepers', 'TFT6_Augment_FuturePeepers2', 'TFT6_Augment_GrandGambler', 'TFT6_Augment_HighEndShopping', 'TFT6_Augment_HyperRoll', 'TFT6_Augment_ItemGrabBag1', 'TFT6_Augment_ItemGrabBag2', 'TFT6_Augment_JeweledLotus', 'TFT6_Augment_LudensEcho1', 'TFT6_Augment_LudensEcho2', 'TFT6_Augment_LudensEcho3', 'TFT6_Augment_MakeshiftArmor1', 'TFT6_Augment_MakeshiftArmor2', 'TFT6_Augment_MakeshiftArmor3', 'TFT6_Augment_MaxLevel10', 'TFT6_Augment_Meditation1', 'TFT6_Augment_Meditation2', 'TFT6_Augment_Meditation3', 'TFT6_Augment_MetabolicAccelerator', 'TFT6_Augment_PandorasItems', 'TFT6_Augment_PortableForge', 'TFT6_Augment_RadiantRelics', 'TFT6_Augment_Recombobulator', 'TFT6_Augment_RichGetRicher', 'TFT6_Augment_RichGetRicherPlus', 'TFT6_Augment_SalvageBin', 'TFT6_Augment_SecondWind1', 'TFT6_Augment_SecondWind2', 'TFT6_Augment_SlowAndSteady', 'TFT6_Augment_SunfireBoard', 'TFT6_Augment_TargetDummies', 'TFT6_Augment_TheGoldenEgg', 'TFT6_Augment_TheGoldenEggHR', 'TFT6_Augment_ThreesCompany', 'TFT6_Augment_ThriftShop', 'TFT6_Augment_ThrillOfTheHunt1', 'TFT6_Augment_ThrillOfTheHunt2', 'TFT6_Augment_TinyTitans', 'TFT6_Augment_TomeOfTraits1', 'TFT6_Augment_TradeSector', 'TFT6_Augment_TradeSectorPlus', 'TFT6_Augment_Traitless1', 'TFT6_Augment_Traitless2', 'TFT6_Augment_Traitless3', 'TFT6_Augment_TriForce1', 'TFT6_Augment_TriForce2', 'TFT6_Augment_TriForce3', 'TFT6_Augment_TrueTwos', 'TFT6_Augment_Twins1', 'TFT6_Augment_Twins2', 'TFT6_Augment_Twins3', 'TFT6_Augment_VerdantVeil', 'TFT6_Augment_Weakspot', 'TFT6_Augment_Windfall', 'TFT6_Augment_WindfallPlus', 'TFT6_Augment_WindfallPlusPlus', 'TFT7_Anivia', 'TFT7_Ashe', 'TFT7_Augment_AFK', 'TFT7_Augment_AFKHR', 'TFT7_Augment_AssassinCutthroat', 'TFT7_Augment_AssassinEmblem', 'TFT7_Augment_AssassinEmblem2', 'TFT7_Augment_AssassinTrait', 'TFT7_Augment_AstralIntercosmicGifts', 'TFT7_Augment_AstralTrait', 'TFT7_Augment_AstralTrait2', 'TFT7_Augment_AxiomArc1', 'TFT7_Augment_AxiomArc2', 'TFT7_Augment_BandOfThieves1', 'TFT7_Augment_BestFriends1', 'TFT7_Augment_BestFriends2', 'TFT7_Augment_BestFriends3', 'TFT7_Augment_BigFriend', 'TFT7_Augment_BigFriend2', 'TFT7_Augment_Bloodlust1', 'TFT7_Augment_BruiserEmblem', 'TFT7_Augment_BruiserEmblem2', 'TFT7_Augment_BruiserPersonalTrainerHR', 'TFT7_Augment_BruiserPersonalTraining', 'TFT7_Augment_BruiserTitanicStrength', 'TFT7_Augment_BruiserTrait', 'TFT7_Augment_CannoneerEmblem', 'TFT7_Augment_CannoneerEmblem2', 'TFT7_Augment_CannoneerHotShot', 'TFT7_Augment_CannoneerRicochet', 'TFT7_Augment_CannoneerTrait', 'TFT7_Augment_CavalierDevastatingCharge', 'TFT7_Augment_CavalierEmblem', 'TFT7_Augment_CavalierEmblem2', 'TFT7_Augment_CavalierForAllUnits', 'TFT7_Augment_CavalierTrait', 'TFT7_Augment_ClutteredMind', 'TFT7_Augment_CursedCrown', 'TFT7_Augment_DragonAlliance', 'TFT7_Augment_DragonHorde', 'TFT7_Augment_DragonmancerEmblem', 'TFT7_Augment_DragonmancerEmblem2', 'TFT7_Augment_DragonmancerInspire', 'TFT7_Augment_DragonmancerTrait', 'TFT7_Augment_EvokerEmblem', 'TFT7_Augment_EvokerEmblem2', 'TFT7_Augment_EvokerEssenceTheft', 'TFT7_Augment_EvokerTrait', 'TFT7_Augment_FirstAidKit2', 'TFT7_Augment_GadgetExpert', 'TFT7_Augment_GuardianEmblem', 'TFT7_Augment_GuardianEmblem2', 'TFT7_Augment_GuardianHeroicPresence', 'TFT7_Augment_GuardianTrait', 'TFT7_Augment_GuildEmblem', 'TFT7_Augment_GuildEmblem2', 'TFT7_Augment_GuildGearUpgrades', 'TFT7_Augment_GuildLoot', 'TFT7_Augment_GuildLootHR', 'TFT7_Augment_GuildTrait', 'TFT7_Augment_JadeEmblem', 'TFT7_Augment_JadeEternalProtection', 'TFT7_Augment_JadePenitence', 'TFT7_Augment_JadeTrait', 'TFT7_Augment_JadeTrait2', 'TFT7_Augment_LastStand', 'TFT7_Augment_LategameSpecialist', 'TFT7_Augment_LivingForge', 'TFT7_Augment_LivingForgeHR', 'TFT7_Augment_MageConference', 'TFT7_Augment_MageConferenceHR', 'TFT7_Augment_MageEmblem', 'TFT7_Augment_MageEmblem2', 'TFT7_Augment_MageTrait', 'TFT7_Augment_MikaelsGift', 'TFT7_Augment_MirageEmblem', 'TFT7_Augment_MirageEmblem2', 'TFT7_Augment_MirageHallucinate', 'TFT7_Augment_MirageTrait', 'TFT7_Augment_MysticTrait', 'TFT7_Augment_MysticTrait2', 'TFT7_Augment_PandorasBench', 'TFT7_Augment_Preparation', 'TFT7_Augment_Preparation1HR', 'TFT7_Augment_Preparation2', 'TFT7_Augment_Preparation2HR', 'TFT7_Augment_Preparation3', 'TFT7_Augment_Preparation3HR', 'TFT7_Augment_RagewingEmblem', 'TFT7_Augment_RagewingEmblem2', 'TFT7_Augment_RagewingScorch', 'TFT7_Augment_RagewingTantrum', 'TFT7_Augment_RagewingTrait', 'TFT7_Augment_RevelEmblem2', 'TFT7_Augment_RevelPartyFavors', 'TFT7_Augment_RevelPartyFavorsHR', 'TFT7_Augment_RevelPartyTime', 'TFT7_Augment_RevelTrait', 'TFT7_Augment_SacrificialPact', 'TFT7_Augment_ScalescornEmblem', 'TFT7_Augment_ScalescornEmblem2', 'TFT7_Augment_ScalescornTrait', 'TFT7_Augment_ShapeshifterBeastsDen', 'TFT7_Augment_ShapeshifterTrait', 'TFT7_Augment_ShapeshifterTrait2', 'TFT7_Augment_ShimmerscaleEmblem', 'TFT7_Augment_ShimmerscaleSpending', 'TFT7_Augment_ShimmerscaleTrait', 'TFT7_Augment_ShimmerscaleTrait2', 'TFT7_Augment_SwiftshotEmblem', 'TFT7_Augment_SwiftshotEmblem2', 'TFT7_Augment_SwiftshotPressTheAttack', 'TFT7_Augment_SwiftshotTrait', 'TFT7_Augment_TempestEmblem', 'TFT7_Augment_TempestEmblem2', 'TFT7_Augment_TempestEyeOfTheStorm', 'TFT7_Augment_TempestTrait', 'TFT7_Augment_ThinkFast', 'TFT7_Augment_TomeOfTraits2', 'TFT7_Augment_UrfsGrabBag1', 'TFT7_Augment_UrfsGrabBag2', 'TFT7_Augment_WarriorEmblem', 'TFT7_Augment_WarriorEmblem2', 'TFT7_Augment_WarriorTiamat', 'TFT7_Augment_WarriorTrait', 'TFT7_Augment_WhispersEmblem2', 'TFT7_Augment_WhispersTrait']
sumkm.to_csv('data/battle_data/data.csv')

NameError: name 'pd' is not defined